In [1]:
import sys, os, time
sys.path.append('../..')
import pyzx as zx
from benchmarking import Benchmark

This notebook demonstrates the results from [Causal flow preserving optimisation of quantum circuits in the ZX-calculus](https://arxiv.org/abs/2312.02793).

In [2]:
# b = Benchmark()
b = Benchmark(dirpath='flow_opt_results')

Circuit metrics are benchmarked against those of Nam, Ross, Su, Childs, Maslov (NRSCM) in [Automated optimization of large quantum circuits with continuous parameters](https://www.nature.com/articles/s41534-018-0072-4).

In [3]:
b.load_circuits(dirname=os.path.join('..', '..', 'circuits', 'benchmarking_circuits', 'Fast', 'before'), group_name='fast')
b.load_circuits(dirname=os.path.join('..', '..', 'circuits', 'benchmarking_circuits', 'Fast', 'nrscm'), group_name='fast', simp_strategy='NRSCM')
b.show_attributes()

Circuit attributes:  ['Qubits', 'Gates', '2Q Count', 'T Count', 't_opt']
Loaded functions:  ['flow-opt-g0', 'flow-opt-c0', 'flow-opt-c1', 'flow-opt-c2', 'flow-opt-c3', 'flow-opt-c4', 'flow-opt-c5']
Loaded routines:  ['NRSCM']
Loaded circuit groups:  ['fast']


,Original,NRSCM,flow-opt-c0,flow-opt-c1,flow-opt-c2,flow-opt-c3,flow-opt-c4,flow-opt-c5,flow-opt-g0
fast,Y,Y,Y,Y,Y,Y,Y,Y,Y


In [4]:
def basic_optimise(c):
    c1 = zx.basic_optimization(c.copy(), do_swaps=False).to_basic_gates()
    c2 = zx.basic_optimization(c.copy(), do_swaps=True).to_basic_gates()
    if c2.twoqubitcount() < c1.twoqubitcount(): return c2 # As this optimisation algorithm is targetted at reducting H-gates, we use the circuit with the smaller 2-qubit gate count here, either using SWAP rules or not.
    return c1

for flow,smax in [('g',0), ('c',0), ('c',1), ('c',2), ('c',3), ('c',4), ('c',5)]:
    def flow_opt(c):
        g = c.to_graph()
        zx.teleport_reduce(g)
        zx.to_graph_like(g)
        zx.flow_2Q_simp(g, cFlow=flow=='c', max_lc_unfusions=smax, max_p_unfusions=smax)
        if flow == 'c': c2 = zx.extract_simple(g).to_basic_gates()
        else: c2 = zx.extract_circuit(g).to_basic_gates()
        return basic_optimise(c2)
    
    b.add_simplification_func(func=flow_opt, name=f'flow-opt-{flow}{smax}', groups_to_run=['fast'], verify=True, rerun=False)

b.show_attributes()

100%|██████████| 31/31 [00:00<00:00, 333393.39it/s]

Circuit attributes:  ['Qubits', 'Gates', '2Q Count', 'T Count', 't_opt']
Loaded functions:  ['flow-opt-g0', 'flow-opt-c0', 'flow-opt-c1', 'flow-opt-c2', 'flow-opt-c3', 'flow-opt-c4', 'flow-opt-c5']
Loaded routines:  ['NRSCM']
Loaded circuit groups:  ['fast']


,Original,NRSCM,flow-opt-c0,flow-opt-c1,flow-opt-c2,flow-opt-c3,flow-opt-c4,flow-opt-c5,flow-opt-g0
fast,Y,Y,Y,Y,Y,Y,Y,Y,Y


In [5]:
df = b.df(groups=['fast'], routines=['all'], funcs=['all'], atts=['Qubits','2Q Count','T Count'])

In [6]:
df = b.df(groups=['fast'], routines=['all'], funcs=['all'], atts=['Qubits','t_opt'])

,Original,flow-opt-c0,flow-opt-c1,flow-opt-c2,flow-opt-c3,flow-opt-c4,flow-opt-c5,flow-opt-g0
,Qubits,t_opt,t_opt,t_opt,t_opt,t_opt,t_opt,t_opt
Circuits,,,,,,,,
Adder8,23,0.44,0.65,1.49,5.16,13.05,30.53,0.28
QFT8,8,0.02,0.03,0.16,1.52,17.24,70.57,0.02
QFTAdd8,16,0.14,0.18,0.79,4.97,18.84,47.90,0.12
adder_8,24,0.57,1.03,4.20,20.07,72.87,207.77,0.46
barenco_tof_10,19,0.20,0.29,0.62,2.14,7.59,21.57,0.18
barenco_tof_3,5,0.01,0.01,0.02,0.03,0.05,0.08,0.01
barenco_tof_4,7,0.02,0.03,0.06,0.18,0.65,1.92,0.02
barenco_tof_5,9,0.04,0.06,0.13,0.48,1.74,4.96,0.03


In [12]:
b.save('flow_opt_results')